In [1]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("./data_2")
docs = loader.load()
docs

[Document(page_content='Course Title\nPython for Data Science and Machine Learning Bootcamp\nThe Data Science Course: Complete Data Science Bootcamp 2024 \nR Programming A-Z™: R For Data Science With Real Exercises! \nStatistics for Data Science and Business Analysis \nData Science A-Z: Hands-On Exercises & ChatGPT Prize [2024] \nMachine Learning, Data Science and Generative AI with Python \nPython A-Z™: Python For Data Science With Real Exercises!\nComplete A.I. & Machine Learning, Data Science Bootcamp \nData Science and Machine Learning Bootcamp with R \nIntroduction to Machine Learning for Data Science \nBuild Responsive Real-World Websites with HTML and CSS \nHTML and CSS for Beginners - Build a Website & Launch ONLINE \nResponsive Web Design: HTML5 + CSS3 for Entrepreneurs 2018 \nWeb Design for Beginners: Real World Coding in HTML & CSS \nHTML5 and CSS3 Fundamentals \nLearn HTML5 Programming From Scratch \nBuild Websites from Scratch with HTML & CSS \nIntroduction to Cloud Comput

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1200, chunk_overlap = 250)
documents = text_splitter.split_documents(docs)

In [3]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [4]:
db = FAISS.from_documents(documents, OllamaEmbeddings())

In [5]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser



In [6]:
from dotenv import load_dotenv
import os

load_dotenv()
groq_api_key=os.getenv('GROQ_API_KEY')


In [7]:
llm = ChatGroq(groq_api_key = groq_api_key,
               model_name="Llama3-8b-8192")

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

prompt=PromptTemplate.from_template(
"""
Act as helpfull Talent Transformation assistant and suggest the best Courses to take based on the given 
context, given {skills} and given {experience}. I want all the output in JSON format.


Prepare 3 sets of courses by total learning hours where Course type is Mandatory:
- Set 1 (1-day plan): Course duration < 3 hours
- Set 2 (3-day plan): Course duration between 3 and 20 hours
- Set 3 (5-day plan): Course duration > 20 hours

Also prepare 1 set of course where Course type is Optional that suits the {experience} and {skills}
- Optional set : Courses should be from Optional course type

For each suggested course, provide the following:
- Course Name
- Course Link
- Course Length
- Learner Level


Please ensure consistency in providing course details and consider the inclusion of mandatory courses in the recommendations.

Suggest the courses only based on the context knowledge
Keep your course recomandation ground in the facts of the context.
If the context does not contain the suitable courses according to the given {skills} then return "No course Found"

Be as concise as possible when giving answers. Do not anounce that you will answering. 


<context>
{context}
<context>

Skills: {skills}

Experience: {experience}

"""
)

In [15]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [16]:
retriever = db.as_retriever()

parser = JsonOutputParser()
setup = RunnableParallel(context=retriever, skills = RunnablePassthrough(), experience = RunnablePassthrough())

chain = setup | prompt | llm

In [17]:
response = chain.invoke({"skills": "python and AI",
              "experience": 2.5})

print(response.content)

Here are the course recommendations in JSON format:

**Mandatory Courses**

**Set 1 (1-day plan): Course duration < 3 hours**
```
[
  {
    "Course Name": "Git for Beginners - Integration with Popular IDEs",
    "Course Link": "https://www.udemy.com/course/git-github-for-beginners-integration-with-popular-ides/?couponCode=LETSLEARNNOWPP",
    "Course Length": 1.5 hours,
    "Learner Level": "Beginner"
  }
]
```

**Set 2 (3-day plan): Course duration between 3 and 20 hours**
```
[
  {
    "Course Name": "Git and GitHub The Complete Git and GitHub Course",
    "Course Link": "https://www.udemy.com/course/git-github-the-complete-git-github-course/?couponCode=LETSLEARNNOWPP",
    "Course Length": 8 hours,
    "Learner Level": "Beginner"
  },
  {
    "Course Name": "Learn DevOps CI/CD with Jenkins using Pipelines and Docker",
    "Course Link": "https://www.udemy.com/course/learn-devops-ci-cd-with-jenkins-using-pipelines-and-docker/",
    "Course Length": 6 hours,
    "Learner Level": "Begi

In [18]:
response_2 = chain.invoke({"skills": "DevOps, SAP BTP",
              "experience": 2})

print(response_2.content)


Here are the suggested courses for each plan:

**Set 1 (1-day plan): Course duration < 3 hours**

* Course 1: "Practical Approach to Version Control Git and GitHub"
	+ Course Link: https://www.udemy.com/course/git-github-for-beginners-integration-with-popular-ides/?couponCode=LETSLEARNNOWPP
	+ Course Length: 2 hours
	+ Learner Level: Beginner
* Course 2: "Linux For Beginners"
	+ Course Link: https://www.udemy.com/course/linux-for-beginners-2021/?couponCode=LETSLEARNNOWPP
	+ Course Length: 2 hours
	+ Learner Level: Beginner

**Set 2 (3-day plan): Course duration between 3 and 20 hours**

* Course 1: "Git & GitHub - The complete step-by-step guide from scratch"
	+ Course Link: https://www.udemy.com/course/git-github-the-complete-git-github-course/?couponCode=LETSLEARNNOWPP
	+ Course Length: 6 hours
	+ Learner Level: Intermediate
* Course 2: "Linux System Administration and Linux Command Line"
	+ Course Link: https://www.udemy.com/course/linux-command-line-volume1/?couponCode=LETSLEARNNOW

In [36]:
prompt_1 = PromptTemplate.from_template(
"""
**Prompt:**

"Act as a helpful Talent Transformation assistant and suggest the best courses to take based on the 
given context and Input.

Prepare 3 sets of courses by total learning hours where Course type is Mandatory:
- Set 1 (1-day plan): Course duration < 3 hours
- Set 2 (3-day plan): Course duration between 3 and 20 hours
- Set 3 (5-day plan): Course duration > 20 hours

Also prepare 1 set of course where Course type is Optional that suits the {experience} and {skills}
- Optional set : Courses should be from Optional course type

**Input:**

* {experience}: A list of integers or a range of years representing the user's experience in their current role or a related field.
* {skills}: A list of strings or categories representing the user's skills and expertise.
* {context}: A dataframe which contains all mandatory and optional courses.


**Output:**

For the given {experience} and {skills} output a JSON object containing three sets for 
recommended courses based on the total learning hours and one set for optional courses. 
In JSON format.

For each suggested course, provide the following:
- Course Name
- Course Link
- Course Length
- Learner Level

**Constraints:**

* Ensure that the recommended courses are sufficiently different from those already completed by the user.
* Prioritize courses with the shortest duration.
* For optional courses, prioritize courses that are more relevant to the user's skills and experience.

**Evaluation Metrics:**

* User experience and skills overlap with recommended course content
* Diversity of recommended courses
* Relevance of recommended courses to user's current role or field
* Practicality and feasibility of the recommended courses

**Model Requirements:**

* Recommend courses based on the user's experience and skills.
* Ensure that recommended courses are relevant to the user's current role or field.
* Consider the user's previous course completions and provide varied and diverse recommendations.

**Evaluation:**

* Precision: How well do the recommended courses align with the user's experience and skills?
* Recall: How well do the recommended courses cover the user's knowledge gaps?
* F1-score: How well-balanced are the recommended courses in terms of relevance and diversity?


<context>
{context}

skills: {skills}

experience: {experience}
"""
)

In [37]:
chain_1 = setup | prompt_1 | llm

In [38]:
response = chain_1.invoke({"user_skills": "Python and AI",
                           "user_experience": 2.5})

print(response.content)

Based on the given context and input, I will suggest three sets of courses by total learning hours where Course type is Mandatory:

**Set 1 (1-day plan): Course duration < 3 hours**

* Course 1: "Learn DevOps with Jenkins and Docker" (2 hours)
	+ Course Link: https://www.udemy.com/course/learn-devops-ci-cd-with-jenkins-using-pipelines-and-docker/
	+ Learner Level: Beginner
* Course 2: "Git for Beginners" (2 hours)
	+ Course Link: https://www.udemy.com/course/git-github-for-beginners-integration-with-popular-ides/
	+ Learner Level: Beginner

**Set 2 (3-day plan): Course duration between 3 and 20 hours**

* Course 1: "The Complete DevOps Bootcamp" (8 hours)
	+ Course Link: https://www.udemy.com/course/the-complete-devops-bootcamp/
	+ Learner Level: Intermediate
* Course 2: "DevOps with Docker and Kubernetes" (6 hours)
	+ Course Link: https://www.udemy.com/course/devops-with-docker-kubernetes-and-azure-devops/
	+ Learner Level: Intermediate
* Course 3: "Learn Terraform Using Docker, AWS, 

In [39]:
response2 = chain_1.invoke({"user_skills": "JavaScript, Python",
                           "user_experience": 2.5})

print(response2.content)

Based on the given input and context, I recommend the following courses:

**Mandatory Courses:**

**Set 1 (1-day plan):**

1. **Course Name:** Learn DevOps CI/CD with Jenkins using Pipelines and Docker
**Course Link:** https://www.udemy.com/course/learn-devops-ci-cd-with-jenkins-using-pipelines-and-docker/
**Course Length:** 1 hour
**Learner Level:** Beginner

**Set 2 (3-day plan):**

1. **Course Name:** DevOps with Docker, Kubernetes, and Azure DevOps
**Course Link:** https://www.udemy.com/course/devops-with-docker-kubernetes-and-azure-devops/
**Course Length:** 6 hours
**Learner Level:** Intermediate

2. **Course Name:** Learn DevOps Infrastructure Automation with Terraform
**Course Link:** https://www.udemy.com/course/learn-devops-infrastructure-automation-with-terraform/
**Course Length:** 4 hours
**Learner Level:** Intermediate

**Set 3 (5-day plan):**

1. **Course Name:** The Complete DevOps Bootcamp
**Course Link:** https://www.udemy.com/course/the-complete-devops-bootcamp/
**Co

In [40]:
response3 = chain_1.invoke({"user_skills": "Machine Learning and AI",
                           "user_experience": 6})

print(response3.content)

Based on the given context and input, I will suggest the best courses to take for the user with skills in Machine Learning and AI and 6 years of experience. Here are the recommended courses:

**Set 1 (1-day plan): Course duration < 3 hours**
```json
[
  {
    "Course Name": "Git and GitHub for Beginners",
    "Course Link": "https://www.udemy.com/course/git-github-for-beginners-integration-with-popular-ides/?couponCode=LETSLEARNNOWPP",
    "Course Length": 1 hour,
    "Learner Level": "Beginner"
  },
  {
    "Course Name": "Linux For Beginners",
    "Course Link": "https://www.udemy.com/course/linux-for-beginners-2021/?couponCode=LETSLEARNNOWPP",
    "Course Length": 2 hours,
    "Learner Level": "Beginner"
  }
]
```
**Set 2 (3-day plan): Course duration between 3 and 20 hours**
```json
[
  {
    "Course Name": "The Ultimate DevOps Bootcamp",
    "Course Link": "https://www.udemy.com/course/the-ultimate-devops-bootcamp-course-for-all-your-devops-learning/?couponCode=LETSLEARNNOWPP",
  

### EX: increasing chunk size to 1300

In [42]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1300, chunk_overlap = 300)
documents = text_splitter.split_documents(docs)

In [43]:
db = FAISS.from_documents(documents, OllamaEmbeddings())

In [44]:
prompt_1 = PromptTemplate.from_template(
"""
**Prompt:**

"Act as a helpful Talent Transformation assistant and suggest the best courses to take based on the 
given context and Input.

Prepare 3 sets of courses by total learning hours where Course type is Mandatory:
- Set 1 (1-day plan): Course duration < 3 hours
- Set 2 (3-day plan): Course duration between 3 and 20 hours
- Set 3 (5-day plan): Course duration > 20 hours

Also prepare 1 set of course where Course type is Optional that suits the {experience} and {skills}
- Optional set : Courses should be from Optional course type

**Input:**

* {experience}: A list of integers or a range of years representing the user's experience in their current role or a related field.
* {skills}: A list of strings or categories representing the user's skills and expertise.
* {context}: A dataframe which contains all mandatory and optional courses.


**Output:**

For the given {experience} and {skills} output a JSON object containing three sets for 
recommended courses based on the total learning hours and one set for optional courses. 
In JSON format.

For each suggested course, provide the following:
- Course Name
- Course Link
- Course Length
- Learner Level

**Constraints:**

* Ensure that the recommended courses are sufficiently different from those already completed by the user.
* Prioritize courses with the shortest duration.
* For optional courses, prioritize courses that are more relevant to the user's skills and experience.

**Evaluation Metrics:**

* User experience and skills overlap with recommended course content
* Diversity of recommended courses
* Relevance of recommended courses to user's current role or field
* Practicality and feasibility of the recommended courses

**Model Requirements:**

* Recommend courses based on the user's experience and skills.
* Ensure that recommended courses are relevant to the user's current role or field.
* Consider the user's previous course completions and provide varied and diverse recommendations.

**Evaluation:**

* Precision: How well do the recommended courses align with the user's experience and skills?
* Recall: How well do the recommended courses cover the user's knowledge gaps?
* F1-score: How well-balanced are the recommended courses in terms of relevance and diversity?


<context>
{context}

skills: {skills}

experience: {experience}
"""
)

In [45]:
chain_1 = setup | prompt_1 | llm

In [49]:
response = chain_1.invoke({"user_skills": "Python and AI",
                           "user_experience": 2.5})

print(response.content)

Based on the given context and input, I will recommend the best courses for the user with skills in Python and AI and 2.5 years of experience. Here are the recommended courses:

**Set 1 (1-day plan): Course duration < 3 hours**

1. **Git and GitHub for Beginners** - https://www.udemy.com/course/git-github-for-beginners-integration-with-popular-ides/
2. **Linux for Beginners** - https://www.udemy.com/course/linux-for-beginners-2021/

**Set 2 (3-day plan): Course duration between 3 and 20 hours**

1. **DevOps Bootcamp** - https://www.udemy.com/course/learn-devops-ci-cd-with-jenkins-using-pipelines-and-docker/
2. **Learn DevOps Infrastructure Automation with Terraform** - https://www.udemy.com/course/learn-devops-infrastructure-automation-with-terraform/
3. **DevOps with Docker, Kubernetes, and Azure DevOps** - https://www.udemy.com/course/devops-with-docker-kubernetes-and-azure-devops/

**Set 3 (5-day plan): Course duration > 20 hours**

1. **The Complete DevOps Bootcamp** - https://www.

In [47]:
response3 = chain_1.invoke({"user_skills": "Machine Learning and AI",
                           "user_experience": 6})

print(response3.content)

Based on the given input, I will suggest three sets of courses with different total learning hours and one set of optional courses. 

**Set 1 (1-day plan): Course duration < 3 hours**

* Course 1: Git and GitHub - The complete step-by-step guide from scratch (Duration: 1 hour)
* Course 2: Practical Approach to Version Control Git and GitHub (Duration: 1 hour)
* Course 3: Linux For Beginners (Duration: 1 hour)

**Set 2 (3-day plan): Course duration between 3 and 20 hours**

* Course 1: The ultimate DevOps Bootcamp course for all your DevOps learning (Duration: 6 hours)
* Course 2: Master Your DevOps Skills with Real challenge (Duration: 8 hours)
* Course 3: DevOps for Absolute Beginners: Azure DevOps, Docker, Kubernetes, Jenkins, Terraform, Ansible - AWS, Azure & Google Cloud (Duration: 10 hours)

**Set 3 (5-day plan): Course duration > 20 hours**

* Course 1: Learn SAP's Business Technology Platform in detail by exploring the Tools and Services available in SAP BTP (Duration: 24 hours)